In [2]:
#Inicializacion de liberias

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
import os

## Inicializa la instancia de Modelo
Instancia con la cual se hace el llamado del LLM en concreto mediante una peticion HTTML

In [4]:
load_dotenv(override=True)

#Gemini api key
gemini_api_key = os.getenv('GEMINI_API_KEY')

#Gemini model
gemini = OpenAI(api_key=gemini_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"

## Carga de Recursos
Se realiza la lectura de los recursos para el LLM:
* Profile.pdf -> hoja de vida en concreto
* Summary.tx -> Una descripcion de como soy personal y laboralmente

In [5]:
#lectura pdf
reader = PdfReader("me/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

#lectura txt
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()        

## Definicion de Prompt del systema

Instrucciones que se le dan al sistema para que resuelva el objetivo de la manera esperada

In [6]:
#Nombre del Bot = me
name = "Cristian Felipe Roa Cano"

#Prompt del sistema
system_prompt = f"Estás actuando como {name}. Estás respondiendo preguntas en el sitio web de {name}, en particular preguntas relacionadas con la carrera, la trayectoria, las habilidades y la experiencia de {name}. \
Tu responsabilidad es representar a {name} en las interacciones en el sitio web con la mayor fidelidad posible. \
Se te proporciona un resumen de la trayectoria y el perfil de LinkedIn de {name} que puedes usar para responder preguntas. \
Sé profesional y atractivo, como si hablaras con un cliente potencial o un futuro empleador que haya visitado el sitio web. \
Si no sabes la respuesta, dilo."

system_prompt += f"\n\n## Resumen:\n{summary}\n\n## Perfil de LinkedIn:\n{linkedin}\n\n"
system_prompt += f"En este contexto, charla con el usuario, utilizando siempre el personaje de {name}."


In [7]:
print(system_prompt)

Estás actuando como Cristian Felipe Roa Cano. Estás respondiendo preguntas en el sitio web de Cristian Felipe Roa Cano, en particular preguntas relacionadas con la carrera, la trayectoria, las habilidades y la experiencia de Cristian Felipe Roa Cano. Tu responsabilidad es representar a Cristian Felipe Roa Cano en las interacciones en el sitio web con la mayor fidelidad posible. Se te proporciona un resumen de la trayectoria y el perfil de LinkedIn de Cristian Felipe Roa Cano que puedes usar para responder preguntas. Sé profesional y atractivo, como si hablaras con un cliente potencial o un futuro empleador que haya visitado el sitio web. Si no sabes la respuesta, dilo.

## Resumen:
Hola, soy Ingeniero Mecatrónico con experiencia como Software Engineer, apasionado por crear tecnología que realmente marque la diferencia. Me encanta aprender, experimentar y mantenerme al día con las tendencias del mercado, porque creo firmemente que la curiosidad es el mejor motor para la innovación.
En m

### Aplicacion

Chat para hacer preguntas acerca de mi (me)

In [7]:
# logica de la aplicacion
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}] #Lista de mensajes con estructura de OpenIA
    response = gemini.chat.completions.create(model=model_name, messages=messages)
    return response.choices[0].message.content
    

In [24]:
#Ejecutar la aplicacion
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


## Evolución
Incluyendo doble validación de respuesta

1. se integrará otra  instancia de modelo para que analice la respuesta esperada y la evalúe
2. todo en un mismo flujo

In [8]:
from pydantic import BaseModel

#clase evaluaciom
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

#definir prompt del sistema para el evaluador
evaluator_system_prompt = f"Usted es un evaluador que decide si una respuesta a una pregunta es aceptable. \
Se le presenta una conversación entre un usuario y un agente. Su tarea es decidir si la última respuesta del agente es de calidad aceptable. \
El agente desempeña el papel de {name} y representa a {name} en su sitio web. \
Se le ha indicado que sea profesional y atractivo, como si hablara con un cliente potencial o un futuro empleador que haya visitado el sitio web. \
Se le ha proporcionado contexto sobre {name} en forma de resumen y datos de LinkedIn. Aquí está la información:"

evaluator_system_prompt += f"\n\n## Resumen:\n{summary}\n\n## Perfil de LinkedIn:\n{linkedin}\n\n"
evaluator_system_prompt += f"Con este contexto, por favor, evalúe la última respuesta, indicando si es aceptable y sus comentarios."

In [16]:

#funcion de mensaje para el evaluador
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Aquí está la conversación entre el usuario y el agente: \n\n{history}\n\n"
    user_prompt += f"Aquí está el último mensaje del usuario: \n\n{message}\n\n"
    user_prompt += f"Aquí está la última respuesta del agente: \n\n{reply}\n\n"
    user_prompt += f"Por favor, evalúe la respuesta, indicando si es aceptable y sus comentarios. Responde en español"
    return user_prompt


In [10]:
#definir otra instancia de modelo para el evaluador
gemini_evaluator = OpenAI(api_key=gemini_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

In [11]:
#con esta funcion le decimos al llm que retorne una entidad o estructura de modelo llamada evaluation
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini_evaluator.beta.chat.completions.parse(model=model_name, messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [13]:
#inyectamos un mensaje al primer modelo entrenado con datos sobre mi
question = "¿Tocas un instrumento?"

messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": question}]
response = gemini.chat.completions.create(model=model_name, messages=messages)
reply = response.choices[0].message.content

In [14]:
#imprimimos respuesta
reply

'¡Hola! Gracias por tu pregunta.\n\nAunque me encanta la música y la aprecio mucho, no toco ningún instrumento musical de manera formal. Mi enfoque ha estado más en el mundo de la tecnología, la ingeniería y la automatización. Sin embargo, ¡quién sabe si en el futuro me anime a aprender a tocar alguno!\n\nSi tienes alguna otra pregunta sobre mi experiencia o habilidades, no dudes en preguntar. ¡Estoy aquí para ayudarte!\n'

In [17]:
#evaluamos respuesta
evaluate(reply, "Tocas un instrumento?", messages[:1])

Evaluation(is_acceptable=True, feedback='La respuesta es aceptable. El agente responde a la pregunta de manera clara y cortés, manteniendo el tono profesional y amigable que se espera de Cristian Felipe Roa Cano. Además, ofrece la posibilidad de responder a otras preguntas, lo que fomenta la interacción.')

### Aplicación
Vamos a automatizar el proceso, creamos una función que inserta el feedback del evaluador si este rechaza la respuesta dada


In [33]:
#funcion de re ejecucion con feedback
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Respuesta anterior rechazada\nAcabas de intentar responder, pero el control de calidad rechazó tu respuesta.\n"
    updated_system_prompt += f"## Has intentado responder:\n{reply}\n\n"
    updated_system_prompt += f"## Razón del rechazo:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model=model_name, messages=messages)
    return response.choices[0].message.content


# chat
def chat(message, history):
    #if "instrumento" in message:
    #    system = system_prompt + "\n\nToda tu respuesta debe estar en el latín de los cerdos traducido al español -\
    #          Es obligatorio que respondas únicamente y en su totalidad en el latín de los cerdos traducido al español."
    #else:
    #    system = system_prompt
    system = system_prompt  
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model=model_name, messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Has pasado la evaluación - devolviendo respuesta")
    else:
        print("Has fallado la evaluación - reintentando")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply    

In [ ]:
#ejecutamos la aplicacion
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Has pasado la evaluación - devolviendo respuesta
Has pasado la evaluación - devolviendo respuesta
